In [1]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.preprocessing.image import ImageDataGenerator

In [2]:
datagen = ImageDataGenerator(rescale=1./255,
                            validation_split = 0.1,rotation_range=30,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

In [3]:
train_ds  = datagen.flow_from_directory(
        'devnagri-script-classification/Data/Train',
        target_size=(100, 100),
        batch_size=1024,
        subset='training',
        class_mode='categorical')

Found 70380 images belonging to 46 classes.


In [4]:
val_ds  = datagen.flow_from_directory(
         'devnagri-script-classification/Data/Train',
        target_size=(100, 100),
        batch_size=1024,
        subset='validation',
        class_mode='categorical')

Found 7820 images belonging to 46 classes.


In [5]:
img, lab = next(iter(train_ds ))
img.shape, lab.shape

((1024, 100, 100, 3), (1024, 46))

In [6]:
class_indices = train_ds.class_indices
len(class_indices)

46

In [7]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense,Activation,BatchNormalization,GlobalAveragePooling2D
from keras.layers import MaxPooling2D
from tensorflow.keras import Model, Input
model = Sequential()

#1 conv layer
model.add(Conv2D(filters = 16, kernel_size = 5, activation = "relu",input_shape=(100,100,3),padding="same"))

#1 max pool layer
model.add(MaxPooling2D(pool_size = 3))

#2 conv layer
model.add(Conv2D(filters = 32, kernel_size = 5, activation = "relu",padding="same"))

#2 max pool layer
model.add(MaxPooling2D(pool_size = 3))

#3 conv layer
model.add(Conv2D(filters = 64, kernel_size = 5, activation = "relu",padding="same"))

#3 max pool layer
model.add(MaxPooling2D(pool_size = 3))

#4 conv layer
model.add(Conv2D(filters = 128, kernel_size = 5, activation = "relu",padding="same"))

# 5 conv layer 
model.add(Conv2D(filters = 256, kernel_size = 5, activation = "relu",padding="same"))

#4 max pool layer
model.add(MaxPooling2D(pool_size = 3))

model.add(BatchNormalization())

model.add(Flatten())


#1 dense layer
model.add(Dense(1000, input_shape = (100,100,3), activation = "relu"))

model.add(Dropout(0.3))

model.add(BatchNormalization())

#2 dense layer
model.add(Dense(500, activation = "relu"))

model.add(Dropout(0.3))

model.add(BatchNormalization())

#3 dense layer 
model.add(Dense(100, activation = "relu"))

model.add(Dropout(0.3))

model.add(BatchNormalization())

#output layer
model.add(Dense(46,activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 16)      1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 33, 33, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 33, 33, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)         2

In [8]:
model.compile(optimizer="RMSprop",loss='categorical_crossentropy',metrics=['accuracy'])

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss',patience=5)

batch_size=100
epochs=10

history = model.fit(train_ds,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_ds,
                    callbacks = [early_stop])

Epoch 1/10
69/69 [==============================] - 328s 5s/step - loss: 3.2279 - accuracy: 0.1683 - val_loss: 3.7406 - val_accuracy: 0.0293
Epoch 2/10
69/69 [==============================] - 321s 5s/step - loss: 2.0178 - accuracy: 0.4363 - val_loss: 3.7102 - val_accuracy: 0.0601
Epoch 3/10
69/69 [==============================] - 354s 5s/step - loss: 1.3791 - accuracy: 0.6028 - val_loss: 3.6532 - val_accuracy: 0.1059
Epoch 4/10
69/69 [==============================] - 325s 5s/step - loss: 1.0108 - accuracy: 0.7051 - val_loss: 3.2303 - val_accuracy: 0.1325
Epoch 5/10
69/69 [==============================] - 323s 5s/step - loss: 0.7808 - accuracy: 0.7706 - val_loss: 2.4394 - val_accuracy: 0.2647
Epoch 6/10
69/69 [==============================] - 327s 5s/step - loss: 0.6270 - accuracy: 0.8160 - val_loss: 1.3376 - val_accuracy: 0.5834
Epoch 7/10
69/69 [==============================] - 332s 5s/step - loss: 0.5334 - accuracy: 0.8433 - val_loss: 1.0854 - val_accuracy: 0.6802
Epoch 8/10
69

In [10]:
test_image = tf.io.gfile.listdir('devnagri-script-classification/Data/Test')
images = []
for f in test_image:
    img = tf.keras.preprocessing.image.load_img('devnagri-script-classification/Data/Test'+'//'+ f, color_mode = "rgb", target_size = (100,100))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img=img/255
    images.append(img)
test=np.array(images)

In [1]:
pred = model.predict(test)
pred

NameError: name 'model' is not defined

In [13]:
pred=np.argmax(pred,axis=1)

In [14]:
class_indices = dict([(value, key) for key, value in class_indices.items()])

pred = list(map(class_indices.get, pred))

In [15]:
submission1 = pd.DataFrame({'id': test_image,'category': pred})
submission1.tail()

,id,category
13795,9995.png,character_33_ha
13796,9996.png,character_7_chha
13797,9997.png,character_7_chha
13798,9998.png,character_7_chha
13799,9999.png,character_7_chha


In [16]:
submission1.to_csv("submission1.csv",index=False)